In [27]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold

In [28]:
sampleSubmission = pd.read_csv('./data/sampleSubmission.csv')
allTrainingDataT0 = pd.read_csv('./data/train_t0.csv')
allTrainingDataT0 = allTrainingDataT0.loc[:, 'f1':'f595']
allTrainingDataT1 = pd.read_csv('./data/train_t1.csv')
allTrainingDataT1 = allTrainingDataT1.loc[:, 'f1':'f595']
finalTestingDataT0 = pd.read_csv('./data/test_t0.csv') #Do not touch it
finalTestingDataT0 = finalTestingDataT0.loc[:, 'f1':'f595']
finalTestingDataT0.head(10)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f586,f587,f588,f589,f590,f591,f592,f593,f594,f595
0,0.122661,0.109112,0.231773,0,0.122661,0.109112,0.092518,0.215179,0.016593,0.092518,...,0.083222,0.155292,0.176474,0.163567,0.098551,0.085389,0.086820,0.004597,0.004597,0.211176
1,0.045889,0.077398,0.031509,0,0.045889,0.077398,0.181591,0.135702,0.104193,0.181591,...,0.087444,0.107125,0.168047,0.123742,0.098362,0.105015,0.099260,0.002731,0.002731,0.180088
2,0.020038,0.026596,0.046635,0,0.020038,0.026596,0.114418,0.134456,0.087822,0.114418,...,0.189319,0.125778,0.253501,0.186712,0.242613,0.193321,0.157509,0.090372,0.090372,0.326458
3,0.015838,0.073108,0.057270,0,0.015838,0.073108,0.112882,0.097044,0.039774,0.112882,...,0.072407,0.051349,0.140081,0.098872,0.092207,0.122251,0.062600,0.002926,0.002926,0.234741
4,0.007235,0.048022,0.040787,0,0.007235,0.048022,0.135871,0.128635,0.087848,0.135871,...,0.057658,0.070526,0.123818,0.139617,0.132032,0.120361,0.061331,0.002288,0.002288,0.213473
5,0.019183,0.094985,0.114168,0,0.019183,0.094985,0.121323,0.140506,0.026339,0.121323,...,0.094795,0.127945,0.237968,0.119351,0.115340,0.104275,0.091191,0.009266,0.009266,0.218438
6,0.106997,0.157994,0.050997,0,0.106997,0.157994,0.117481,0.010484,0.040513,0.117481,...,0.137664,0.026861,0.313305,0.229986,0.131211,0.168520,0.157771,0.042123,0.042123,0.332688
7,0.005972,0.023589,0.017617,0,0.005972,0.023589,0.178104,0.172131,0.154515,0.178104,...,0.076848,0.048370,0.222110,0.157774,0.158509,0.135718,0.123088,0.022291,0.022291,0.303554
8,0.081350,0.067428,0.013921,0,0.081350,0.067428,0.128558,0.047208,0.061129,0.128558,...,0.093422,0.115975,0.139737,0.137801,0.083554,0.097807,0.115830,0.001051,0.001051,0.204185
9,0.053963,0.107713,0.161676,0,0.053963,0.107713,0.110151,0.164114,0.002438,0.110151,...,0.184799,0.018664,0.176090,0.187941,0.119456,0.169442,0.177676,0.062520,0.062520,0.328674


In [29]:
#Remove outliers by mean method 
np.warnings.filterwarnings('ignore')
from scipy import stats
threshold= 2 #Work best at 2
mean_outliered_allTrainingDataT0 = allTrainingDataT0
mean_outliered_allTrainingDataT1 = allTrainingDataT1
for i in range(len(mean_outliered_allTrainingDataT0.loc[0,:])):
    feature = "f"+str(i+1)
    meanT0 = mean_outliered_allTrainingDataT0[feature].to_numpy().mean()
    meanT1 = mean_outliered_allTrainingDataT1[feature].to_numpy().mean()
    zT0=np.abs(stats.zscore(mean_outliered_allTrainingDataT0[feature]))
    for j in range(len(np.where(zT0>threshold)[0])):
        mean_outliered_allTrainingDataT0.loc[np.where(zT0>threshold)[0][j],[feature]] = meanT0
    zT1=np.abs(stats.zscore(mean_outliered_allTrainingDataT1[feature]))
    for j in range(len(np.where(zT1>threshold)[0])):
        mean_outliered_allTrainingDataT1.loc[np.where(zT1>threshold)[0][j],[feature]] = meanT1
        

print(mean_outliered_allTrainingDataT0.shape)

(150, 595)


In [30]:
#Remove outliers by neighbour method

np.warnings.filterwarnings('ignore')
from scipy import stats
threshold=3
neighbour_outliered_allTrainingDataT0 = allTrainingDataT0
neighbour_outliered_allTrainingDataT1 = allTrainingDataT1
for i in range(len(neighbour_outliered_allTrainingDataT0)):
    feature = "f"+str(i+1)
    zT0=np.abs(stats.zscore(neighbour_outliered_allTrainingDataT0[feature]))
    for j in range(len(np.where(zT0>threshold)[0])):
        if(np.where(zT0>threshold)[0][j] == 149):
            next_index = np.where(zT0>threshold)[0][j]-2
            previous_index = np.where(zT0>threshold)[0][j]-1
        elif(np.where(zT0>threshold)[0][j] == 0):
            next_index = np.where(zT0>threshold)[0][j]+1
            previous_index = np.where(zT0>threshold)[0][j]+2
        else:
            next_index = np.where(zT0>threshold)[0][j]+1
            previous_index = np.where(zT0>threshold)[0][j]-1
            
        neighbour_outliered_allTrainingDataT0.loc[np.where(zT0>threshold)[0][j],[feature]] = (neighbour_outliered_allTrainingDataT0.loc[next_index,[feature]] + neighbour_outliered_allTrainingDataT0.loc[previous_index,[feature]])/2

    zT1=np.abs(stats.zscore(neighbour_outliered_allTrainingDataT1[feature]))
    for j in range(len(np.where(zT1>threshold)[0])):
        if(np.where(zT1>threshold)[0][j] == 149):
            next_index = np.where(zT1>threshold)[0][j]-2
            previous_index = np.where(zT1>threshold)[0][j]-1
        elif(np.where(zT1>threshold)[0][j] == 0):
            next_index = np.where(zT1>threshold)[0][j]+1
            previous_index = np.where(zT1>threshold)[0][j]+2
        else:
            next_index = np.where(zT1>threshold)[0][j]+1
            previous_index = np.where(zT1>threshold)[0][j]-1
        neighbour_outliered_allTrainingDataT1.loc[np.where(zT1>threshold)[0][j],[feature]] = (neighbour_outliered_allTrainingDataT1.loc[next_index,[feature]] + neighbour_outliered_allTrainingDataT1.loc[previous_index,[feature]])/2
        

print(neighbour_outliered_allTrainingDataT0.shape)


(150, 595)


In [31]:
"""
#Scale the data ----- Not worked well
normalized_changedDataT0 = (changedDataT0-changedDataT0.mean())/changedDataT0.std()
normalized_changedDataT1 = (changedDataT1-changedDataT1.mean())/changedDataT1.std()


normalized_changedDataT0 = pd.DataFrame(np.nan_to_num(normalized_changedDataT0))
normalized_changedDataT1 = pd.DataFrame(np.nan_to_num(normalized_changedDataT1))
print("DONE")
"""

'\n#Scale the data ----- Not worked well\nnormalized_changedDataT0 = (changedDataT0-changedDataT0.mean())/changedDataT0.std()\nnormalized_changedDataT1 = (changedDataT1-changedDataT1.mean())/changedDataT1.std()\n\n\nnormalized_changedDataT0 = pd.DataFrame(np.nan_to_num(normalized_changedDataT0))\nnormalized_changedDataT1 = pd.DataFrame(np.nan_to_num(normalized_changedDataT1))\nprint("DONE")\n'

In [32]:
#5 Fold Cross Validation
from sklearn.multioutput import MultiOutputRegressor

numberOfFolds = 5
kf = KFold(n_splits=numberOfFolds, shuffle=True) 
split_indices = kf.split(range(allTrainingDataT0.shape[0]))

regr = MultiOutputRegressor(SVR(kernel='rbf', C=1000, gamma=.1, epsilon=.01))
allFoldsErrors = []
cv_index = 1
for train_indices, test_indices in split_indices:
    X = allTrainingDataT0.loc[train_indices, :]
    Y = allTrainingDataT1.loc[train_indices, :]
    
    regr.fit(X, Y)
    prediction = regr.predict(allTrainingDataT0.loc[test_indices, :])
    meanSquaredError = mse(allTrainingDataT1.loc[test_indices, :].to_numpy().flatten(), prediction.flatten())
    print("\n\n\nCV Fold {} : Mean Squared Error -> {:.5f}\n".format(cv_index, meanSquaredError))
    allFoldsErrors.append(meanSquaredError)
    cv_index += 1
   
    """
    PRINT Indicies and Data
    
    print("\nThese are train indicies: {}".format(train_indices))
    print("\nThis is training data:\n {}\n\n".format(allTainingDataT0.loc[train_indices, :]))
    print("\nThese are test indices: {}".format(test_indices))
    print("\nThis is testing data:\n {}\n\n".format(allTainingDataT0.loc[test_indices, :]))
    """
    
print("Average Error => {:.5f}".format(sum(allFoldsErrors) / numberOfFolds))




CV Fold 1 : Mean Squared Error -> 0.00240




CV Fold 2 : Mean Squared Error -> 0.00191




CV Fold 3 : Mean Squared Error -> 0.00230




CV Fold 4 : Mean Squared Error -> 0.00214




CV Fold 5 : Mean Squared Error -> 0.00225

Average Error => 0.00220


In [33]:
regr.fit(mean_outliered_allTrainingDataT0, mean_outliered_allTrainingDataT1)
prediction = regr.predict(finalTestingDataT0)
predictionFlattened = prediction.flatten()

submission = [["ID","predicted"]]
for i in range(47600):
    submission.append([i, predictionFlattened[i]])

submissionDF = pd.DataFrame(submission)
submissionDF.to_csv('submission.csv', index=False, header=False)
print("DONE")

DONE


In [34]:
readSubmission = pd.read_csv('./submission.csv')
readSubmission.head()

,ID,predicted
0,0,0.027712
1,1,0.103964
2,2,0.145209
3,3,0.000000
4,4,0.027712
